# Table Building

## Setup

In [1]:
import numpy as np
import pandas as pd
import re
import json

## Read Data

In [2]:
with open('../data/cleaned_data.json') as f:
    data = json.load(f)
f.close()

## Build `song_lib` table
This might be useful for getting song info later, but ultimately our library will be based on albums as being the "books" with songs as "chapters"

### Initialize dictionary we'll use to make the data frame later

In [3]:
song_lib_dict = dict()

### Loop over json and add songs and metadata to `lib_dict` with an arbitrary song ID

In [4]:
idx = 0
# Loop over artists
for artist in data.keys():
    # Loop over albums
    for album in data[artist]['Albums'].keys():
        # Loop over tracks
        for track in data[artist]['Albums'][album]['tracklist'].keys():
            song_lib_dict[idx] = {
                'title':track,
                'json_path':(f'data[{artist}][\'Albums\'][{album}][\'tracklist\'][track]'),
                'artist':artist,
                'album':album,
                'genres':data[artist]['Metadata']['genres'],
                'release_date':data[artist]['Albums'][album]['release_date'],
                'label':data[artist]['Albums'][album]['label'],
                'track_number':data[artist]['Albums'][album]['tracklist'][track]['track_number'],
                'duration_ms':data[artist]['Albums'][album]['tracklist'][track]['duration_ms'],
                'track_number':data[artist]['Albums'][album]['tracklist'][track]['track_number'],
                'audio_information':data[artist]['Albums'][album]['tracklist'][track]['audio_information'],
                'lyrics':data[artist]['Albums'][album]['tracklist'][track]['lyrics']
            }
            idx += 1

### Convert `lib_dict` to data frame

In [5]:
song_lib = pd.DataFrame.from_dict(song_lib_dict).T
song_lib.index.name = 'song_id'

In [6]:
song_lib.head()

,title,json_path,artist,album,genres,release_date,label,track_number,duration_ms,audio_information,lyrics
song_id,,,,,,,,,,,
0,Pay Your Way In Pain,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,1,183880,"{'danceability': 0.677, 'energy': 0.826, 'key'...",\n(Ow)\nOh-oh-oh\n\nYou got to pay your way in...
1,Down And Out Downtown,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,2,222240,"{'danceability': 0.61, 'energy': 0.502, 'key':...",\nLast night's heels\nOn the mornin' train\nIt...
2,Daddy's Home,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,3,199613,"{'danceability': 0.774, 'energy': 0.372, 'key'...",\nI signed autographs in the visitation room\n...
3,Live In The Dream,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,4,389546,"{'danceability': 0.426, 'energy': 0.339, 'key'...",\nHello\nDo you know where you are?\nYou've be...
4,The Melting Of The Sun,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,5,257519,"{'danceability': 0.52, 'energy': 0.463, 'key':...","\nSo sorry, missed the party\nHello, on the da..."


### Expand `audio_information` to individual columns

In [7]:
audio_info = song_lib.audio_information

In [8]:
audio_frame = audio_info.apply(pd.Series)

In [9]:
audio_frame.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
song_id,,,,,,,,,,,,,,,,,,
0,0.677,0.826,9,-6.800,1,0.3680,0.3950,0.00000,0.5710,0.4980,124.821,audio_features,1nLaoWVhNRZ38irY0LZjL6,spotify:track:1nLaoWVhNRZ38irY0LZjL6,https://api.spotify.com/v1/tracks/1nLaoWVhNRZ3...,https://api.spotify.com/v1/audio-analysis/1nLa...,183880,4
1,0.610,0.502,6,-8.409,1,0.0820,0.0578,0.01410,0.1660,0.6730,151.906,audio_features,4htB9ArZCpquXlXnkKjAgk,spotify:track:4htB9ArZCpquXlXnkKjAgk,https://api.spotify.com/v1/tracks/4htB9ArZCpqu...,https://api.spotify.com/v1/audio-analysis/4htB...,222240,4
2,0.774,0.372,11,-9.796,0,0.0956,0.6150,0.00148,0.1060,0.6020,110.036,audio_features,6DeE4f5DFeWycZz8f6UEht,spotify:track:6DeE4f5DFeWycZz8f6UEht,https://api.spotify.com/v1/tracks/6DeE4f5DFeWy...,https://api.spotify.com/v1/audio-analysis/6DeE...,199613,4
3,0.426,0.339,2,-10.262,1,0.0270,0.7510,0.06450,0.0871,0.0689,80.224,audio_features,27JieQ9wlcYMf1SwTczPUl,spotify:track:27JieQ9wlcYMf1SwTczPUl,https://api.spotify.com/v1/tracks/27JieQ9wlcYM...,https://api.spotify.com/v1/audio-analysis/27Ji...,389547,4
4,0.520,0.463,10,-10.429,1,0.0958,0.6090,0.00140,0.4260,0.4690,159.931,audio_features,3WD3w5uSzAGJWrNFnHhi1a,spotify:track:3WD3w5uSzAGJWrNFnHhi1a,https://api.spotify.com/v1/tracks/3WD3w5uSzAGJ...,https://api.spotify.com/v1/audio-analysis/3WD3...,257520,4


#### Drop some redundant or unnecessary columns

In [10]:
audio_frame.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [11]:
audio_frame = audio_frame.drop(['type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms'], axis=1)

In [12]:
audio_frame.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
song_id,,,,,,,,,,,,
0,0.677,0.826,9,-6.800,1,0.3680,0.3950,0.00000,0.5710,0.4980,124.821,4
1,0.610,0.502,6,-8.409,1,0.0820,0.0578,0.01410,0.1660,0.6730,151.906,4
2,0.774,0.372,11,-9.796,0,0.0956,0.6150,0.00148,0.1060,0.6020,110.036,4
3,0.426,0.339,2,-10.262,1,0.0270,0.7510,0.06450,0.0871,0.0689,80.224,4
4,0.520,0.463,10,-10.429,1,0.0958,0.6090,0.00140,0.4260,0.4690,159.931,4


#### Add `audio_frame` to `lib`

In [13]:
song_lib = song_lib.join(audio_frame)

In [14]:
song_lib.head()

,title,json_path,artist,album,genres,release_date,label,track_number,duration_ms,audio_information,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
song_id,,,,,,,,,,,,,,,,,,,,,
0,Pay Your Way In Pain,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,1,183880,"{'danceability': 0.677, 'energy': 0.826, 'key'...",...,9,-6.800,1,0.3680,0.3950,0.00000,0.5710,0.4980,124.821,4
1,Down And Out Downtown,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,2,222240,"{'danceability': 0.61, 'energy': 0.502, 'key':...",...,6,-8.409,1,0.0820,0.0578,0.01410,0.1660,0.6730,151.906,4
2,Daddy's Home,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,3,199613,"{'danceability': 0.774, 'energy': 0.372, 'key'...",...,11,-9.796,0,0.0956,0.6150,0.00148,0.1060,0.6020,110.036,4
3,Live In The Dream,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,4,389546,"{'danceability': 0.426, 'energy': 0.339, 'key'...",...,2,-10.262,1,0.0270,0.7510,0.06450,0.0871,0.0689,80.224,4
4,The Melting Of The Sun,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,5,257519,"{'danceability': 0.52, 'energy': 0.463, 'key':...",...,10,-10.429,1,0.0958,0.6090,0.00140,0.4260,0.4690,159.931,4


### Change `dtypes` to reasonable types

In [15]:
song_lib.dtypes

title                 object
json_path             object
artist                object
album                 object
genres                object
release_date          object
label                 object
track_number          object
duration_ms           object
audio_information     object
lyrics                object
danceability         float64
energy               float64
key                    int64
loudness             float64
mode                   int64
speechiness          float64
acousticness         float64
instrumentalness     float64
liveness             float64
valence              float64
tempo                float64
time_signature         int64
dtype: object

In [16]:
song_lib = song_lib.astype({
    'track_number':'int16',
    'duration_ms':'int32'
})

In [17]:
song_lib.dtypes

title                 object
json_path             object
artist                object
album                 object
genres                object
release_date          object
label                 object
track_number           int16
duration_ms            int32
audio_information     object
lyrics                object
danceability         float64
energy               float64
key                    int64
loudness             float64
mode                   int64
speechiness          float64
acousticness         float64
instrumentalness     float64
liveness             float64
valence              float64
tempo                float64
time_signature         int64
dtype: object

## Build `lib` table
This is based on albums as "books" and songs as "chapters"

In [18]:
LIB = song_lib.groupby('album').agg({
    'artist':'first',
    'genres':'first',
    'release_date':'first',
    'label':'first',
    'danceability':'mean',
    'energy':'mean',
    'loudness':'mean',
    'speechiness':'mean',
    'acousticness':'mean',
    'instrumentalness':'mean',
    'liveness':'mean',
    'valence':'mean',
    'tempo':'mean'
})

In [19]:
LIB = LIB.rename(columns={
    'artist':'artist',
    'genres':'genres',
    'release_date':'release_date',
    'label':'label',
    'danceability':'mean_danceability',
    'energy':'mean_energy',
    'loudness':'mean_loudness',
    'speechiness':'mean_speechiness',
    'acousticness':'mean_acousticness',
    'instrumentalness':'mean_instrumentalness',
    'liveness':'mean_liveness',
    'valence':'mean_valence',
    'tempo':'mean_tempo'
})

In [20]:
LIB = LIB.sort_index(ascending=True).reset_index()
LIB.index.name = 'album_id'
LIB.head()

,album,artist,genres,release_date,label,mean_danceability,mean_energy,mean_loudness,mean_speechiness,mean_acousticness,mean_instrumentalness,mean_liveness,mean_valence,mean_tempo
album_id,,,,,,,,,,,,,,
0,45:33,LCD Soundsystem,"[alternative dance, alternative rock, art pop,...",2007-11-12,Parlophone UK,0.625000,0.918000,-5.810000,0.074050,0.177000,0.672500,0.105000,0.456000,104.008500
1,A Moon Shaped Pool,Radiohead,"[alternative rock, art rock, melancholia, oxfo...",2016-05-08,XL Recordings,0.429545,0.406455,-11.596818,0.034264,0.673291,0.564455,0.158773,0.203018,125.244000
2,Actor,St. Vincent,"[art pop, etherpop, indie rock, indietronica, ...",2009-05-05,4AD,0.484091,0.548455,-10.276273,0.039882,0.324571,0.433091,0.214909,0.329773,119.461091
3,Al Mundo Azul,Mr. Twin Sister,[chillwave],2021-11-19,Twin Group,0.722100,0.698500,-6.106500,0.054780,0.275440,0.330606,0.191360,0.716500,125.701400
4,Alligator,The National,"[chamber pop, indie folk, indie rock, indietro...",2005-04-11,Beggars Banquet,0.522154,0.740769,-6.816615,0.035585,0.284209,0.239670,0.125769,0.535831,125.053308


In [21]:
LIB = LIB.rename(columns={'album':'album_title'})
LIB.head()

,album_title,artist,genres,release_date,label,mean_danceability,mean_energy,mean_loudness,mean_speechiness,mean_acousticness,mean_instrumentalness,mean_liveness,mean_valence,mean_tempo
album_id,,,,,,,,,,,,,,
0,45:33,LCD Soundsystem,"[alternative dance, alternative rock, art pop,...",2007-11-12,Parlophone UK,0.625000,0.918000,-5.810000,0.074050,0.177000,0.672500,0.105000,0.456000,104.008500
1,A Moon Shaped Pool,Radiohead,"[alternative rock, art rock, melancholia, oxfo...",2016-05-08,XL Recordings,0.429545,0.406455,-11.596818,0.034264,0.673291,0.564455,0.158773,0.203018,125.244000
2,Actor,St. Vincent,"[art pop, etherpop, indie rock, indietronica, ...",2009-05-05,4AD,0.484091,0.548455,-10.276273,0.039882,0.324571,0.433091,0.214909,0.329773,119.461091
3,Al Mundo Azul,Mr. Twin Sister,[chillwave],2021-11-19,Twin Group,0.722100,0.698500,-6.106500,0.054780,0.275440,0.330606,0.191360,0.716500,125.701400
4,Alligator,The National,"[chamber pop, indie folk, indie rock, indietro...",2005-04-11,Beggars Banquet,0.522154,0.740769,-6.816615,0.035585,0.284209,0.239670,0.125769,0.535831,125.053308


### Create `album_name` column with no spaces to be able to reference files

In [22]:
LIB['album_name'] = LIB['album_title'].apply(lambda x: re.sub(r' ', '_', x))
LIB.head()

,album_title,artist,genres,release_date,label,mean_danceability,mean_energy,mean_loudness,mean_speechiness,mean_acousticness,mean_instrumentalness,mean_liveness,mean_valence,mean_tempo,album_name
album_id,,,,,,,,,,,,,,,
0,45:33,LCD Soundsystem,"[alternative dance, alternative rock, art pop,...",2007-11-12,Parlophone UK,0.625000,0.918000,-5.810000,0.074050,0.177000,0.672500,0.105000,0.456000,104.008500,45:33
1,A Moon Shaped Pool,Radiohead,"[alternative rock, art rock, melancholia, oxfo...",2016-05-08,XL Recordings,0.429545,0.406455,-11.596818,0.034264,0.673291,0.564455,0.158773,0.203018,125.244000,A_Moon_Shaped_Pool
2,Actor,St. Vincent,"[art pop, etherpop, indie rock, indietronica, ...",2009-05-05,4AD,0.484091,0.548455,-10.276273,0.039882,0.324571,0.433091,0.214909,0.329773,119.461091,Actor
3,Al Mundo Azul,Mr. Twin Sister,[chillwave],2021-11-19,Twin Group,0.722100,0.698500,-6.106500,0.054780,0.275440,0.330606,0.191360,0.716500,125.701400,Al_Mundo_Azul
4,Alligator,The National,"[chamber pop, indie folk, indie rock, indietro...",2005-04-11,Beggars Banquet,0.522154,0.740769,-6.816615,0.035585,0.284209,0.239670,0.125769,0.535831,125.053308,Alligator


### Create `source_file_path` column

In [23]:
LIB['source_file_path'] = LIB.album_name.apply(lambda x: f'../data/albums/{x}.txt')
LIB.head()

,album_title,artist,genres,release_date,label,mean_danceability,mean_energy,mean_loudness,mean_speechiness,mean_acousticness,mean_instrumentalness,mean_liveness,mean_valence,mean_tempo,album_name,source_file_path
album_id,,,,,,,,,,,,,,,,
0,45:33,LCD Soundsystem,"[alternative dance, alternative rock, art pop,...",2007-11-12,Parlophone UK,0.625000,0.918000,-5.810000,0.074050,0.177000,0.672500,0.105000,0.456000,104.008500,45:33,../data/albums/45:33.txt
1,A Moon Shaped Pool,Radiohead,"[alternative rock, art rock, melancholia, oxfo...",2016-05-08,XL Recordings,0.429545,0.406455,-11.596818,0.034264,0.673291,0.564455,0.158773,0.203018,125.244000,A_Moon_Shaped_Pool,../data/albums/A_Moon_Shaped_Pool.txt
2,Actor,St. Vincent,"[art pop, etherpop, indie rock, indietronica, ...",2009-05-05,4AD,0.484091,0.548455,-10.276273,0.039882,0.324571,0.433091,0.214909,0.329773,119.461091,Actor,../data/albums/Actor.txt
3,Al Mundo Azul,Mr. Twin Sister,[chillwave],2021-11-19,Twin Group,0.722100,0.698500,-6.106500,0.054780,0.275440,0.330606,0.191360,0.716500,125.701400,Al_Mundo_Azul,../data/albums/Al_Mundo_Azul.txt
4,Alligator,The National,"[chamber pop, indie folk, indie rock, indietro...",2005-04-11,Beggars Banquet,0.522154,0.740769,-6.816615,0.035585,0.284209,0.239670,0.125769,0.535831,125.053308,Alligator,../data/albums/Alligator.txt


### Add `song_regex` column

In [24]:
LIB['song_regex'] = '\[Trackname:\s[^\]]+\]'

In [25]:
LIB.head()

,album_title,artist,genres,release_date,label,mean_danceability,mean_energy,mean_loudness,mean_speechiness,mean_acousticness,mean_instrumentalness,mean_liveness,mean_valence,mean_tempo,album_name,source_file_path,song_regex
album_id,,,,,,,,,,,,,,,,,
0,45:33,LCD Soundsystem,"[alternative dance, alternative rock, art pop,...",2007-11-12,Parlophone UK,0.625000,0.918000,-5.810000,0.074050,0.177000,0.672500,0.105000,0.456000,104.008500,45:33,../data/albums/45:33.txt,\[Trackname:\s[^\]]+\]
1,A Moon Shaped Pool,Radiohead,"[alternative rock, art rock, melancholia, oxfo...",2016-05-08,XL Recordings,0.429545,0.406455,-11.596818,0.034264,0.673291,0.564455,0.158773,0.203018,125.244000,A_Moon_Shaped_Pool,../data/albums/A_Moon_Shaped_Pool.txt,\[Trackname:\s[^\]]+\]
2,Actor,St. Vincent,"[art pop, etherpop, indie rock, indietronica, ...",2009-05-05,4AD,0.484091,0.548455,-10.276273,0.039882,0.324571,0.433091,0.214909,0.329773,119.461091,Actor,../data/albums/Actor.txt,\[Trackname:\s[^\]]+\]
3,Al Mundo Azul,Mr. Twin Sister,[chillwave],2021-11-19,Twin Group,0.722100,0.698500,-6.106500,0.054780,0.275440,0.330606,0.191360,0.716500,125.701400,Al_Mundo_Azul,../data/albums/Al_Mundo_Azul.txt,\[Trackname:\s[^\]]+\]
4,Alligator,The National,"[chamber pop, indie folk, indie rock, indietro...",2005-04-11,Beggars Banquet,0.522154,0.740769,-6.816615,0.035585,0.284209,0.239670,0.125769,0.535831,125.053308,Alligator,../data/albums/Alligator.txt,\[Trackname:\s[^\]]+\]


### Reorder columns

In [26]:
LIB = LIB[[
    'album_name',
    'album_title',
    'artist',
    'source_file_path',
    'song_regex',
    'genres',
    'release_date',
    'label',
    'mean_danceability',
    'mean_energy',
    'mean_loudness',
    'mean_speechiness',
    'mean_acousticness',
    'mean_instrumentalness',
    'mean_liveness',
    'mean_valence',
    'mean_tempo'
]]

In [27]:
LIB.head()

,album_name,album_title,artist,source_file_path,song_regex,genres,release_date,label,mean_danceability,mean_energy,mean_loudness,mean_speechiness,mean_acousticness,mean_instrumentalness,mean_liveness,mean_valence,mean_tempo
album_id,,,,,,,,,,,,,,,,,
0,45:33,45:33,LCD Soundsystem,../data/albums/45:33.txt,\[Trackname:\s[^\]]+\],"[alternative dance, alternative rock, art pop,...",2007-11-12,Parlophone UK,0.625000,0.918000,-5.810000,0.074050,0.177000,0.672500,0.105000,0.456000,104.008500
1,A_Moon_Shaped_Pool,A Moon Shaped Pool,Radiohead,../data/albums/A_Moon_Shaped_Pool.txt,\[Trackname:\s[^\]]+\],"[alternative rock, art rock, melancholia, oxfo...",2016-05-08,XL Recordings,0.429545,0.406455,-11.596818,0.034264,0.673291,0.564455,0.158773,0.203018,125.244000
2,Actor,Actor,St. Vincent,../data/albums/Actor.txt,\[Trackname:\s[^\]]+\],"[art pop, etherpop, indie rock, indietronica, ...",2009-05-05,4AD,0.484091,0.548455,-10.276273,0.039882,0.324571,0.433091,0.214909,0.329773,119.461091
3,Al_Mundo_Azul,Al Mundo Azul,Mr. Twin Sister,../data/albums/Al_Mundo_Azul.txt,\[Trackname:\s[^\]]+\],[chillwave],2021-11-19,Twin Group,0.722100,0.698500,-6.106500,0.054780,0.275440,0.330606,0.191360,0.716500,125.701400
4,Alligator,Alligator,The National,../data/albums/Alligator.txt,\[Trackname:\s[^\]]+\],"[chamber pop, indie folk, indie rock, indietro...",2005-04-11,Beggars Banquet,0.522154,0.740769,-6.816615,0.035585,0.284209,0.239670,0.125769,0.535831,125.053308


## Build `CORPUS` table
Using the following OHCO:
[album_id, song_id, stanza_num, line_num, token_num]

This will require a bit of work given we store all of the lyrics at a song level in the .json

### Label song lyrics with song name
For simplicity we also go ahead and make these individual `.txt` files for each album

In [28]:
# Loop over artists
# for artist in data.keys():
#     # Loop over albums
#     for album in data[artist]['Albums'].keys():
#         album_text = ''
#         album_name = re.sub(r' ', '_', album)
#         # Loop over songs
#         for track in data[artist]['Albums'][album]['tracklist'].keys():
#             lyrics = data[artist]['Albums'][album]['tracklist'][track]['lyrics']
#             lyrics = f'\n\n[Trackname: {track}]\n' + lyrics
#             album_text  = album_text + lyrics
#         with open(f'../data/albums/{album_name}.txt', 'w') as outfile:
#             print(f'Wrote album {album_name} to file {album_name}.txt')
#             print(f'{album_text}', file=outfile)
            

## Parse album `.txt` files into `CORPUS` table

<3 Raf for `textimporter.py`